In [1]:
import pyspark as ps
from pyspark.sql import SQLContext
from pyspark.sql import Row
import datetime, time
from pyspark.sql import functions as func
from pyspark.sql.functions import col

#  Create SparkContext

In [2]:
sc

In [3]:
#sconf = ps.SparkConf().setAppName("WeblogChallenge").setMaster("local[*]")

In [4]:
#sc = ps.SparkContext(conf=sconf)

#   Read the file

In [5]:
input = sc.textFile("file:///Users/gopasing/WeblogChallenge/data/2015_07_22_mktplace_shop_web_log_sample.log.gz")

In [6]:
input.take(3)

['2015-07-22T09:00:28.019143Z marketpalce-shop 123.242.248.130:54635 10.0.6.158:80 0.000022 0.026109 0.00002 200 200 0 699 "GET https://paytm.com:443/shop/authresponse?code=f2405b05-e2ee-4b0d-8f6a-9fed0fcfe2e0&state=null HTTP/1.1" "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/43.0.2357.130 Safari/537.36" ECDHE-RSA-AES128-GCM-SHA256 TLSv1.2',
 '2015-07-22T09:00:27.894580Z marketpalce-shop 203.91.211.44:51402 10.0.4.150:80 0.000024 0.15334 0.000026 200 200 0 1497 "GET https://paytm.com:443/shop/wallet/txnhistory?page_size=10&page_number=0&channel=web&version=2 HTTP/1.1" "Mozilla/5.0 (Windows NT 6.1; rv:39.0) Gecko/20100101 Firefox/39.0" ECDHE-RSA-AES128-GCM-SHA256 TLSv1.2',
 '2015-07-22T09:00:27.885745Z marketpalce-shop 1.39.32.179:56419 10.0.4.244:80 0.000024 0.164958 0.000017 200 200 0 157 "GET https://paytm.com:443/shop/wallet/txnhistory?page_size=10&page_number=0&channel=web&version=2 HTTP/1.1" "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHT

In [7]:
#schema = "evtDt website hostIP sessionIP website num1 num2 num3 num4 num5 num6 num7 API browser sessionID prot"


In [8]:
wrkData = input.map(lambda row:row.split('"')).map(lambda row:(row[0].split(' '),row[1],row[2],row[3],row[4].strip()))

In [9]:
wrkData.take(3)

[(['2015-07-22T09:00:28.019143Z',
   'marketpalce-shop',
   '123.242.248.130:54635',
   '10.0.6.158:80',
   '0.000022',
   '0.026109',
   '0.00002',
   '200',
   '200',
   '0',
   '699',
   ''],
  'GET https://paytm.com:443/shop/authresponse?code=f2405b05-e2ee-4b0d-8f6a-9fed0fcfe2e0&state=null HTTP/1.1',
  ' ',
  'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/43.0.2357.130 Safari/537.36',
  'ECDHE-RSA-AES128-GCM-SHA256 TLSv1.2'),
 (['2015-07-22T09:00:27.894580Z',
   'marketpalce-shop',
   '203.91.211.44:51402',
   '10.0.4.150:80',
   '0.000024',
   '0.15334',
   '0.000026',
   '200',
   '200',
   '0',
   '1497',
   ''],
  'GET https://paytm.com:443/shop/wallet/txnhistory?page_size=10&page_number=0&channel=web&version=2 HTTP/1.1',
  ' ',
  'Mozilla/5.0 (Windows NT 6.1; rv:39.0) Gecko/20100101 Firefox/39.0',
  'ECDHE-RSA-AES128-GCM-SHA256 TLSv1.2'),
 (['2015-07-22T09:00:27.885745Z',
   'marketpalce-shop',
   '1.39.32.179:56419',
   '10.0.4.244:80',
   

In [10]:
wrkDataRow = wrkData.map(lambda row:Row(
        evtUnixDt=int(time.mktime(datetime.datetime.strptime(row[0][0],"%Y-%m-%dT%H:%M:%S.%fZ").timetuple())),
        website=row[0][1],
        userIP=row[0][2],
        hostIP=row[0][3],
        num1=row[0][4],
        num2=row[0][5],
        num3=row[0][6],
        api=row[1],
        browser=row[3],
        sessionId= row[4]
    ))

In [11]:
wrkDataRow.take(3)

[Row(api='GET https://paytm.com:443/shop/authresponse?code=f2405b05-e2ee-4b0d-8f6a-9fed0fcfe2e0&state=null HTTP/1.1', browser='Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/43.0.2357.130 Safari/537.36', evtUnixDt=1437570028, hostIP='10.0.6.158:80', num1='0.000022', num2='0.026109', num3='0.00002', sessionId='ECDHE-RSA-AES128-GCM-SHA256 TLSv1.2', userIP='123.242.248.130:54635', website='marketpalce-shop'),
 Row(api='GET https://paytm.com:443/shop/wallet/txnhistory?page_size=10&page_number=0&channel=web&version=2 HTTP/1.1', browser='Mozilla/5.0 (Windows NT 6.1; rv:39.0) Gecko/20100101 Firefox/39.0', evtUnixDt=1437570027, hostIP='10.0.4.150:80', num1='0.000024', num2='0.15334', num3='0.000026', sessionId='ECDHE-RSA-AES128-GCM-SHA256 TLSv1.2', userIP='203.91.211.44:51402', website='marketpalce-shop'),
 Row(api='GET https://paytm.com:443/shop/wallet/txnhistory?page_size=10&page_number=0&channel=web&version=2 HTTP/1.1', browser='Mozilla/5.0 (Windows NT 6.1

#  Create the workData DataFrame

In [12]:
wrkDataDF=sqlContext.createDataFrame(wrkDataRow)

In [13]:
wrkDataDF.show(4)

+--------------------+--------------------+----------+-------------+--------+--------+--------+--------------------+--------------------+----------------+
|                 api|             browser| evtUnixDt|       hostIP|    num1|    num2|    num3|           sessionId|              userIP|         website|
+--------------------+--------------------+----------+-------------+--------+--------+--------+--------------------+--------------------+----------------+
|GET https://paytm...|Mozilla/5.0 (Wind...|1437570028|10.0.6.158:80|0.000022|0.026109| 0.00002|ECDHE-RSA-AES128-...|123.242.248.130:5...|marketpalce-shop|
|GET https://paytm...|Mozilla/5.0 (Wind...|1437570027|10.0.4.150:80|0.000024| 0.15334|0.000026|ECDHE-RSA-AES128-...| 203.91.211.44:51402|marketpalce-shop|
|GET https://paytm...|Mozilla/5.0 (Wind...|1437570027|10.0.4.244:80|0.000024|0.164958|0.000017|ECDHE-RSA-AES128-...|   1.39.32.179:56419|marketpalce-shop|
|GET https://paytm...|                   -|1437570028|10.0.6.108:80| 0

#  1. Sessionize by user IP - Number of "15 mins(900 secs) sessions" within the session duration...

In [14]:
sessionDF = wrkDataDF.select("sessionId","userIP","evtUnixDt").filter("sessionId LIKE '%SHA%'")

In [15]:
sessionDF.show(1000)

+--------------------+--------------------+----------+
|           sessionId|              userIP| evtUnixDt|
+--------------------+--------------------+----------+
|ECDHE-RSA-AES128-...|123.242.248.130:5...|1437570028|
|ECDHE-RSA-AES128-...| 203.91.211.44:51402|1437570027|
|ECDHE-RSA-AES128-...|   1.39.32.179:56419|1437570027|
|ECDHE-RSA-AES128-...|180.179.213.94:48725|1437570028|
|ECDHE-RSA-AES128-...|120.59.192.208:13527|1437570028|
|ECDHE-RSA-AES128-...|117.239.195.66:50524|1437570028|
|ECDHE-RSA-AES128-...| 101.60.186.26:33177|1437570028|
|ECDHE-RSA-AES128-...|  59.183.41.47:62014|1437570028|
|ECDHE-RSA-AES128-...|117.239.195.66:50538|1437570028|
|ECDHE-RSA-AES128-...| 183.83.237.83:49687|1437570028|
|ECDHE-RSA-AES128-...| 117.195.91.36:25902|1437570028|
|ECDHE-RSA-AES128-...|122.180.245.251:5...|1437570028|
|ECDHE-RSA-AES128-...|117.198.215.20:53517|1437570028|
|ECDHE-RSA-AES128-...|223.176.154.91:51582|1437570028|
|ECDHE-RSA-AES128-...|223.225.236.110:3...|1437570028|
|ECDHE-RSA

In [16]:
userIPsessionDF = sessionDF.filter("userIP != '-'").groupBy("userIP").agg((func.max("evtUnixDt")-func.min("evtUnixDt")).alias("duration"))

In [17]:
sessionizeuserIPsessionDF = userIPsessionDF.select("userIP",(col("duration")/900).alias("NumOfSessions"))

In [18]:
sessionizeuserIPsessionDF.show(100)

+--------------------+--------------------+
|              userIP|       NumOfSessions|
+--------------------+--------------------+
|59.177.155.238:23325|                 0.0|
| 115.110.71.22:33063|0.001111111111111...|
|122.179.36.196:11739|0.001111111111111...|
|  52.74.219.71:56506|   6.027777777777778|
|203.99.196.126:58737|                 0.0|
| 54.251.151.39:44334|                 0.0|
|164.100.136.87:61075|0.017777777777777778|
| 117.239.11.194:2773| 0.08333333333333333|
|122.177.183.66:58829|0.001111111111111...|
| 171.51.61.221:50800|0.017777777777777778|
|  202.12.83.44:50300|                 0.0|
|  170.61.236.1:51014| 0.08333333333333333|
|164.100.149.245:3...|0.002222222222222...|
| 59.145.196.114:2273|                 0.0|
| 122.182.14.98:49027|0.056666666666666664|
|150.129.130.221:5...|0.017777777777777778|
| 106.77.231.26:49611| 0.10111111111111111|
|203.92.217.240:46790|  0.1388888888888889|
|    103.5.132.5:2552| 0.06666666666666667|
| 115.112.73.34:49771|0.00444444

#  Create the dataframe to get each session duration

In [19]:
grpsessionDF=sessionDF.groupBy("sessionId").agg((func.max("evtUnixDt")-func.min("evtUnixDt")).alias("duration"))

In [20]:
grpsessionDF.show(4,False)

+-----------------------------------+--------+
|sessionId                          |duration|
+-----------------------------------+--------+
|ECDHE-RSA-AES128-GCM-SHA256 TLSv1.2|66621   |
|ECDHE-RSA-AES128-SHA TLSv1.1       |66619   |
|ECDHE-RSA-AES128-SHA TLSv1.2       |66589   |
|DES-CBC3-SHA TLSv1.2               |21263   |
+-----------------------------------+--------+
only showing top 4 rows



#    2. Average session Time

In [21]:
avgSessionTime=grpsessionDF.agg(func.avg("duration"))

In [22]:
avgSessionTime.show()

+-----------------+
|    avg(duration)|
+-----------------+
|51465.21428571428|
+-----------------+



#    3. unique URL visits per session

In [23]:
apiwrkDataDF = wrkDataDF.select("api","sessionId").filter("sessionId LIKE '%SHA%'")

In [24]:
apiwrkDataDF.show(4,False)

+----------------------------------------------------------------------------------------------------------------+-----------------------------------+
|api                                                                                                             |sessionId                          |
+----------------------------------------------------------------------------------------------------------------+-----------------------------------+
|GET https://paytm.com:443/shop/authresponse?code=f2405b05-e2ee-4b0d-8f6a-9fed0fcfe2e0&state=null HTTP/1.1       |ECDHE-RSA-AES128-GCM-SHA256 TLSv1.2|
|GET https://paytm.com:443/shop/wallet/txnhistory?page_size=10&page_number=0&channel=web&version=2 HTTP/1.1      |ECDHE-RSA-AES128-GCM-SHA256 TLSv1.2|
|GET https://paytm.com:443/shop/wallet/txnhistory?page_size=10&page_number=0&channel=web&version=2 HTTP/1.1      |ECDHE-RSA-AES128-GCM-SHA256 TLSv1.2|
|GET https://paytm.com:443/shop/p/micromax-yu-yureka-moonstone-grey-MOBMICROMAX-YU-DUMM141CD60

In [25]:
uniqueapiwrkDataDF = apiwrkDataDF.groupBy("sessionId").agg(func.countDistinct("api"))

In [26]:
uniqueapiwrkDataDF.show(100,False)

+-----------------------------------+----------+
|sessionId                          |count(api)|
+-----------------------------------+----------+
|ECDHE-RSA-AES128-GCM-SHA256 TLSv1.2|169366    |
|ECDHE-RSA-AES128-SHA TLSv1.1       |16971     |
|ECDHE-RSA-AES128-SHA TLSv1.2       |718       |
|DES-CBC3-SHA TLSv1.2               |88        |
|DHE-RSA-AES128-SHA TLSv1.1         |37        |
|DHE-RSA-AES128-SHA TLSv1.2         |202       |
|ECDHE-RSA-AES128-SHA256 TLSv1.2    |8477      |
|DHE-RSA-AES128-SHA TLSv1           |18444     |
|ECDHE-RSA-AES128-SHA TLSv1         |15675     |
|AES128-SHA TLSv1                   |952       |
|DES-CBC3-SHA TLSv1                 |259       |
|AES128-SHA TLSv1.1                 |2         |
|AES128-SHA TLSv1.2                 |302       |
|AES128-SHA256 TLSv1.2              |67        |
+-----------------------------------+----------+



#    4. Most Engaged user

In [27]:
userwrkDataDF = wrkDataDF.select("userIP","evtUnixDt")

In [28]:
grpuserwrkDataDF = userwrkDataDF.groupBy("userIP").agg((func.max("evtUnixDt")-func.min("evtUnixDt")).alias("duration"))

In [29]:
mostEngagedgrpuserwrkDataDF = grpuserwrkDataDF.sort(col("duration").desc())

In [30]:
mostEngagedgrpuserwrkDataDF.show(15,False)

+-------------------+--------+
|userIP             |duration|
+-------------------+--------+
|106.186.23.95:35626|66511   |
|106.186.23.95:35629|66511   |
|106.186.23.95:35632|66511   |
|106.186.23.95:40452|66500   |
|106.186.23.95:39568|66500   |
|106.186.23.95:39958|66500   |
|106.186.23.95:39950|66500   |
|106.186.23.95:40448|66500   |
|106.186.23.95:40510|66500   |
|106.186.23.95:40515|66500   |
|106.186.23.95:39172|66500   |
|106.186.23.95:40275|66500   |
|106.186.23.95:39944|66500   |
|106.186.23.95:39320|66500   |
|106.186.23.95:39628|66500   |
+-------------------+--------+
only showing top 15 rows

